### 절대 모멘텀
1. 파생변수 'STD-YM' 생성 -> 인덱스에서 년-월을 추출하여 대입 
2. 'STD-YM' 별 마지막날의 데이터들을 month_last_df 추가 (단순 행 결합)
3. 전월의 수정 종가 값을 가진 파생변수 생성 
4. 전년도의 수정 종가 값을 가진 파생변수 생성 
5. 전월의 데이터와 전년도의 데이터를 이용하여 거래 내역 생성 
6. 수익율 계산

In [ ]:
import pandas as pd
import numpy as np 
from datetime import datetime
import warnings

In [ ]:
# 위험 메시지 출력 필터
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("../../csv/AMZN.csv")

In [ ]:
if 'Date' in df.columns:
    df.set_index('Date', inplace=True)

In [ ]:
# index를 시계열로 전환
df.index = pd.to_datetime(df.index)

In [ ]:
# df에 STD-YM 컬럼을 생성하여 인덱스에서 년도-월 데이터를 추출하여 대입
df['STD-YM'] = df.index.strftime('%Y-%m')

In [ ]:
# 빈 데이터프레임에 월말의 데이터들을 행결합 
# STD-YM에서 다음 행과 현재 행이 다르다면
flag = df['STD-YM'] != df['STD-YM'].shift(-1)
df.loc[flag, ]

In [ ]:
# 기준년월의 유니크 값을 따로 생성 
month_list = df['STD-YM'].unique()
month_list

In [ ]:
month_last_df = pd.DataFrame()

for month in month_list:
    # flag = df['STD-YM'] == month
    # data = df.loc[flag, ].tail(1)
    data = df.loc[month, ].tail(1)
    month_last_df = pd.concat(
        [month_last_df, data], axis=0
    )

In [ ]:
month_last_df = month_last_df[['Adj Close', 'STD-YM']]
month_last_df.head()

In [ ]:
# 전월의 수정종가 파생변수(BF-1M) 생성
month_last_df['BF-1M'] = month_last_df['Adj Close'].shift(1).fillna(0)
# 전년도의 수정종가 파생변수(BF-12M) 생성
month_last_df['BF-12M'] = month_last_df['Adj Close'].shift(12).fillna(0)

In [ ]:
month_last_df

- 거래 내역 추가 
    - (전월의 수정주가) / (전년도의 수정주가) - 1 값이 0보다 크고 무한대가 아닌 경우가 매수 타이밍
    - 모멘텀 인덱스가 위의 조건에 부합할 경우 df의 trade 컬럼에 해당 월에 구매 내역 추가 

In [ ]:
df['trade'] = ""

# month_last_df의 인덱스를 기준으로 반복문 실행 
for idx in month_last_df.index:
    signal = ""

    # 절대 모멘텀 인덱스를 계산 
    momentum_index = (month_last_df.loc[idx, 'BF-1M'] / 
                    month_last_df.loc[idx, 'BF-12M']) - 1
    # print(momentum_index)
    # break
    # 조건식 : 모멘텀 인덱스가 0보다 크고 무한대가 아닌
    # 참인 경우 데이터 if 조건식 else 거짓인 경우 데이터
    flag = True if (momentum_index > 0) & (momentum_index != np.inf) \
        else False

    if flag:
        signal = 'buy'
    print(f"날짜 : {idx}, 모멘텀 인덱스 : {momentum_index}, signal : {signal}")
    df.loc[idx:, 'trade'] = signal

In [ ]:
df['trade'].value_counts()

In [ ]:
# 수익율 계산
# 매수 일자 : 전날의 trade가 ""이고 오늘의 trade "buy"
# 매도 일자 : 전날의 trade가 "buy"이고 오늘의 trade ""

df['rtn'] = 1

for idx in df.index:
    # 매수의 조건식 
    if (df.shift().loc[idx, 'trade'] == "") & \
        (df.loc[idx, 'trade'] == "buy"):
        buy = df.loc[idx, 'Adj Close']
        print(f"매수일 : {idx}, 매수가 : {buy}")
    # 매도의 조건식
    elif (df.shift().loc[idx, 'trade'] == 'buy') & \
        (df.loc[idx, 'trade'] == ""):
        sell = df.loc[idx, 'Adj Close']
        rtn = sell / buy
        df.loc[idx, 'rtn'] = rtn
        print(f"매도일 : {idx}, 매도가 : {sell}, 수익율 : {rtn}")

In [ ]:
df['acc_rtn'] = df['rtn'].cumprod()

In [ ]:
acc_rtn = 1
for i in df.index:
    rtn = df.loc[i, 'rtn']
    acc_rtn *= rtn
acc_rtn

In [ ]:
df.iloc[-1, ]['Adj Close'] / df.iloc[0, ]['Adj Close']

In [ ]:
df2 = df.drop(
    ['trade', 'rtn', 'acc_rtn'], 
    axis = 1
)

In [ ]:
df2['trade'] = ""

# month_last_df의 인덱스를 기준으로 반복문 실행 
for idx in month_last_df.index:
    # 절대 모멘텀 인덱스를 계산 
    momentum_index = (month_last_df.loc[idx, 'BF-1M'] / 
                    month_last_df.loc[idx, 'BF-12M']) - 1

    flag = True if (momentum_index > 0) & (momentum_index != np.inf) \
        else False

    if flag:
        # # 모멘텀 인덱스가 구매 신호 주면 해당 월말부터 
        # # 마지막 데이터까지 buy 채워준다.
        # df2.loc[idx:, 'trade'] = 'buy'
        # 모멘텀 인덱스가 구매 신호를 주면 해당 월말 다음 행의 STD-YM 추출하여
        # 해당 조건 맞는 인덱스를 필터링 하여 trade에 buy을 대입 
        std_ym = df2.shift(-1).loc[idx, 'STD-YM']
        df2.loc[std_ym, 'trade'] = 'buy'
        print(f"날짜 : {idx}, 모멘텀 인덱스 : {momentum_index}")
    

In [ ]:
df2['trade'].value_counts()

In [ ]:
# 수익율 계산
# 매수 일자 : 전날의 trade가 ""이고 오늘의 trade "buy"
# 매도 일자 : 전날의 trade가 "buy"이고 오늘의 trade ""

df2['rtn'] = 1

for idx in df2.index:
    # 매수의 조건식 
    if (df2.shift().loc[idx, 'trade'] == "") & \
        (df2.loc[idx, 'trade'] == "buy"):
        buy = df2.loc[idx, 'Adj Close']
        print(f"매수일 : {idx}, 매수가 : {buy}")
    # 매도의 조건식
    elif (df2.shift().loc[idx, 'trade'] == 'buy') & \
        (df2.loc[idx, 'trade'] == ""):
        sell = df2.loc[idx, 'Adj Close']
        rtn = sell / buy
        df2.loc[idx, 'rtn'] = rtn
        print(f"매도일 : {idx}, 매도가 : {sell}, 수익율 : {rtn}")

In [ ]:
df2['rtn'].cumprod()

### 절대 모멘텀 함수화
1. STD-YM 컬럼을 생성하는 함수 (create_YM)
    - 매개변수 2개 
        - 데이터프레임 (_df) : 필수 
        - 기준이 되는 컬럼 : 'Adj Close' 기본값
    1. 데이터프레임 복사본 생성 
    2. 컬럼에 Date가 존재하면 index로 변경 
    3. index를 시계열로 변경 
    4. 결측치나 무한대 값들은 제거 
    5. 기준이 되는 컬럼을 제외하고 나머지는 제거 
    6. 'STD-YM' 컬럼을 생성하여 인덱스에서 년도-월 데이터를 추출하여 대입
    7. 수정이 된 데이터프레임을 되돌려준다. 

In [44]:
def create_YM(
    _df, 
    _col = 'Adj Close'
):
    df = _df.copy()
    if 'Date' in df.columns:
        df.set_index('Date', inplace=True)
    df.index = pd.to_datetime(df.index)
    flag = df.isin( [np.nan, np.inf, -np.inf] ).any(axis=1)
    df = df.loc[~flag, ]
    df = df[[_col]]
    df['STD-YM'] = df.index.strftime('%Y-%m')
    return df

In [45]:
df = pd.read_csv("../../csv/AAPL.csv")

In [47]:
ym_df = create_YM(df)

In [ ]:
ym_df

- 월말의 데이터프레임을 생성하는 함수 (create_last_month)
    - 매개변수 
        - STD-YM 컬럼이 생성된 데이터프레임 (_df) : 필수
        - 투자 시작시간 : 2010-01-01 기본값
        - 투자 종료시간 : 현재 시간 기본값
        - 모멘텀 기간 : 12 기본값
            - 6으로 지정한다면 전월과 6개월 전 데이터를 이용
    - 월말의 데이터만 모아서 새로운 데이터프레임을 생성 
    - 전월의 기준이되는 컬럼의 데이터를 BF1 컬럼에 대입 
    - BF2 컬럼을 생성하여 모멘텀기간만큼 전의 데이터를 대입 
    - 결측치는 0으로 대체
    - 투자의 시작 시간과 종료 시간으로 필터링 
    - 수정된 데이터프레임을 되돌려준다. 

In [49]:
def create_last_month(
    _df, 
    _start = '2010-01-01', 
    _end = datetime.now(), 
    _momentum = 12
):
    # 기준이 되는 컬럼의 이름을 변수에 저장 
    col = _df.columns[0]
    # 월말의 기준 : STD-YM이 현재와 다음행의 데이터가 다른경우
    flag = _df['STD-YM'] != _df.shift(-1)['STD-YM']
    df = _df.loc[flag, ]
    # 전월의 데이터를 생성 
    df['BF1'] = df.shift(1)[col].fillna(0)
    # _momentum의 개월 전의 데이터를 생성 
    df['BF2'] = df.shift(_momentum)[col].fillna(0)
    # 시작 시간과 종료 시간을 기준으로 필터링 
    df = df.loc[_start : _end, ]
    return df

In [52]:
month_df = create_last_month(ym_df)

- 거래 내역을 추가하는 함수 (create_rtn)
    - 매개변수 
        - create_YM() 함수의 결과 (_df1) : 필수
        - create_last_month() 함수의 결과 (_df2) : 필수
        - momentum 스코어 : 1 기본값
    1. _df1의 복사본 생성 (df 생성)
    2. df에 trade 컬럼을 생성해서 "" 대입
    3. _df2의 데이터를 이용해서 momentum_index를 생성 
    4. momentum_index 값에 따라 df 거래 내역을 추가 
    5. df에 rtn 컬럼을 생성해서 1을 대입
    6. df의 trade를 기준으로 수익율을 계산해서 rtn 컬럼에 대입 
    7. 누적 수익율을 계산하여 acc_rtn에 대입
    7. df와 최종 수익율을 되돌려준다. 

In [51]:
def create_rtn(
    _df1, _df2, 
    _score = 1
):
    df = _df1.copy()
    # trade, rtn 컬럼을 생성
    df['trade'] = ""
    df['rtn'] = 1

    # _df2를 이용해서 momentum_index를 구한다. 
    for idx in _df2.index:
        signal = ""

        # 모멘텀 인덱스를 계산 
        momentum_index = \
            _df2.loc[idx, 'BF1'] / _df2.loc[idx, 'BF2'] - _score
        
        flag = (momentum_index > 0) & (momentum_index != np.inf)

        if flag : 
            signal = 'buy'
        # df에 구매내역 추가
        df.loc[idx: , 'trade'] = signal
    # 기준이 되는 컬럼의 이름 변수에 저장 
    col = df.columns[0]

    for idx in df.index:
        # 매수 조건 
        if (df.shift().loc[idx, 'trade'] == "") & \
            (df.loc[idx, 'trade'] == "buy"):
            buy = df.loc[idx, col]
            print(f"매수일 : {idx}, 매수가 : {buy}")
        # 매도 조건
        elif (df.shift().loc[idx, 'trade'] == 'buy') & \
            (df.loc[idx, 'trade'] == ""):
            sell = df.loc[idx, col]
            rtn = sell / buy
            df.loc[idx, 'rtn'] = rtn
            print(f"매도일 : {idx}, 매도가 : {sell}, 수익율 : {rtn}")
    #  누적수익율 계산
    df['acc_rtn'] = df['rtn'].cumprod()
    acc_rtn = df.iloc[-1, -1]
    return df, acc_rtn


In [54]:
create_rtn(ym_df, month_df, 1.1)

매수일 : 2010-01-29 00:00:00, 매수가 : 24.035734
매도일 : 2013-02-28 00:00:00, 매도가 : 56.055256, 수익율 : 2.3321632699047177
매수일 : 2014-01-31 00:00:00, 매수가 : 64.792686
매도일 : 2015-10-30 00:00:00, 매도가 : 111.95108, 수익율 : 1.7278351448495282
매수일 : 2017-01-31 00:00:00, 매수가 : 116.711029
매도일 : 2018-05-31 00:00:00, 매도가 : 184.068115, 수익율 : 1.5771269997113984
매수일 : 2018-06-29 00:00:00, 매수가 : 182.334488
매도일 : 2018-12-31 00:00:00, 매도가 : 156.463837, 수익율 : 0.858114329967022
매수일 : 2019-04-30 00:00:00, 매수가 : 199.900192
매도일 : 2019-05-31 00:00:00, 매도가 : 175.070007, 수익율 : 0.8757870877882898


(             Adj Close   STD-YM trade  rtn  acc_rtn
 Date                                               
 1980-12-12    0.410525  1980-12        1.0  1.00000
 1980-12-15    0.389106  1980-12        1.0  1.00000
 1980-12-16    0.360548  1980-12        1.0  1.00000
 1980-12-17    0.369472  1980-12        1.0  1.00000
 1980-12-18    0.380182  1980-12        1.0  1.00000
 ...                ...      ...   ...  ...      ...
 2019-06-18  198.449997  2019-06        1.0  4.77608
 2019-06-19  197.869995  2019-06        1.0  4.77608
 2019-06-20  199.460007  2019-06        1.0  4.77608
 2019-06-21  198.779999  2019-06        1.0  4.77608
 2019-06-24  199.169998  2019-06        1.0  4.77608
 
 [9713 rows x 5 columns],
 4.776080231784856)